In [3]:
import json

In [4]:
def load_json(file):
    f = open(file, 'r')
    return json.load(f)

In [5]:
def write_json(data):
    encoded_data = json.JSONEncoder(indent=2).encode(data)
    
    f = open('mud_of_babel_data.json', 'w')
    f.write(encoded_data)

In [40]:
def build_basic_game_info():
    return {
        'commandCounter' : 0,
        'gameOver' : False,
        'introText' : '\n\nWelcome to the MUD of Babel!\n',
        'outroText' : 'Thanks For playing!',
        'player' : {
            'currentLocation' : '',
            'inventory' : {},
            'lightSource' : False
        },
        'map': {}
    }

In [41]:
def build_room(parent, info):
    _id = "%s_%s" % (parent, info['id'])
    _subject = info['subject']
    _child_count = len(info['children'])
    exits = {}
    if ('next' in info):
        exits.update({
            'south' : {
                'displayName' : 'South',
                'destination' : "%s_%s" % (parent, info['next'])
            }
        })
    if ('previous' in info):
        exits.update({
            'north' : {
                'displayName' : 'North',
                'destination' : "%s_%s" % (parent, info['previous'])
            }
        })
    room = {
        _id: {
            'firstVisit' : True,
            'displayName' : _subject,
            'description' : "\nThe description for %s\n\n" % _subject,
#             'setup' : 'function(){end();}',
            'interactables' : {
                'helmets' : { 'look' : 'It is a pile of miner helmets with lights on them. They seem to still be operational.' },
                'sign' : { 'look' : 'The sign reads "Crooked Gulch Gold Mine" and has a note tacked to the bottom of it.' },
                'note' : { 'look' : 'Written in an untidy scroll the note reads "Generator blew. Lights out."' }
            },
            'items' : {
                'helmet' : {
                    'displayName' : 'Miner Helmet',
                    'description' : 'A trusty old miner helmet covered in minor dents. Still seems sturdy and the light works.',
#                     'use' : 'function(){return useLightSource();}',
                    'quantity' : 1,
                    'hidden' : True
                }
            },
            'exits' : exits
        }
    }
    return room

In [42]:
def parse_class_file(class_code):
    raw_json = load_json(("%s.json" % class_code))
    subject = raw_json[class_code]['subject']
    game_info = build_basic_game_info()
    rooms = {}
    next_room = None
    keys = list(raw_json[class_code]['children'].keys())
    for key in keys:
        # going south so exits are north
        if (next_room):
            raw_json[class_code]['children'][key]['previous'] = next_room
        next_room = raw_json[class_code]['children'][key]['id']
    keys.reverse()
    for key in keys:
        # going north so exits are south
        if (next_room):
            raw_json[class_code]['children'][key]['next'] = next_room
        next_room = raw_json[class_code]['children'][key]['id']
    for key, value in raw_json[class_code]['children'].items():
        room = build_room(parent=class_code, info=value)
        rooms.update(room)
    first_room = list(rooms.keys())[0]
    game_info['map'] = rooms
    game_info['player']['currentLocation'] = first_room
    write_json(game_info)

parse_class_file('A')